In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from TrainValTensorBoard import TrainValTensorBoard
from time import gmtime, strftime

In [2]:
df_train = pd.read_csv('../data/train_V2.csv')
df_test = pd.read_csv('../data/test_V2.csv')

In [3]:
## BoostedTrees Regressor
# Input Features: KillPlace, WalkDistance
# Labels: WinPlacePerc [probability of 0-1]
# Output: WinPlacePerc

In [4]:
df_rand = df_train.sample(frac=1).reset_index(drop=True)

In [5]:
df_rand.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,0ceb518de3c7b1,07eb8c2e5f6496,88cffe1ae97aff,0,0,0.00,0,0,0,80,...,0,0.0,0,0.0,0,0,51.33,3,0,0.1786
1,aea3061bbcd149,e7d9b1abceb20a,33450f1a2d1e7f,0,3,101.30,0,0,3,37,...,1,6798.0,0,0.0,0,0,2753.00,8,1464,0.8636
2,80475e60f2944d,e41fbcd7a894f2,76a76495bba8c6,0,0,0.17,0,0,0,61,...,0,2060.0,0,0.0,0,0,1189.00,3,1507,0.3571
3,b12ec7ff07834a,ebc0628cb1f5aa,18a66951e93862,0,0,46.89,0,0,0,67,...,0,0.0,0,0.0,0,0,502.70,3,1500,0.3462
4,4fd54434b28abf,8580992571dda7,5f449afba1cf33,0,0,0.00,0,0,0,73,...,0,0.0,0,0.0,0,0,305.00,2,0,0.3846


In [6]:
X_train = df_rand[['killPlace', 'walkDistance']].head(4000000)
Y_train = df_rand[['winPlacePerc']].head(4000000)

In [7]:
X_val = df_rand[['killPlace', 'walkDistance']].tail(446966)
Y_val = df_rand[['winPlacePerc']].tail(446966)

In [8]:
X_test = df_test[['killPlace', 'walkDistance']]

In [9]:
killPlace_fcol = tf.feature_column.numeric_column(
    key='killPlace',
    shape=(1,),
    default_value=None,
    dtype=tf.dtypes.float32,
    normalizer_fn=None)

walkDistance_fcol = tf.feature_column.numeric_column(
    key='walkDistance',
    shape=(1,),
    default_value=None,
    dtype=tf.dtypes.float32,
    normalizer_fn=None
)
feat_cols = [killPlace_fcol, walkDistance_fcol]

In [10]:
# Train input func
# convert to array
x_train = {'killPlace': X_train.values[:,0], 'walkDistance': X_train.values[:,1]}
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x_train,
    y=np.array(Y_train),
    num_epochs=None,
    shuffle=True)

In [11]:
# Validation input func
x_val_in = {'killPlace': X_val.values[:,0], 'walkDistance': X_val.values[:,1]}
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x_val_in,
    y=np.array(Y_val),
    num_epochs=1,
    shuffle=False)

In [12]:
# Prediction input function
x_test_in = {'killPlace': X_test.values[:,0], 'walkDistance': X_test.values[:,1]}
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x_test_in,
    shuffle=False)

In [13]:
# Timestamp for log folder
s = strftime("%a_%d_%b_%Y_%H:%M", gmtime())

In [14]:
#help(tf.estimator.BoostedTreesRegressor)

In [15]:
# BoostedTrees Regressor
# requires: feat coles, n_batches_per_layer
# returns: an instance of a premade estimator class for Boosted trees
bt_regressor = tf.estimator.BoostedTreesRegressor(
    feature_columns=feat_cols,
    n_batches_per_layer=1,
    model_dir='./logs/'+s,
    #label_dimension=_HOLD_FOR_MULTI_DIM_SUPPORT,
    #weight_column=None,
    n_trees=100,
    max_depth=6,
    learning_rate=0.1,
    l1_regularization=0.0,
    l2_regularization=0.0,
    tree_complexity=0.0,
    min_node_weight=0.0,
    #config=None,
    #center_bias=False,
    #pruning_mode='none',
    #quantile_sketch_epsilon=0.01
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './logs/Mon_11_Mar_2019_18:00', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12fb58ac8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [16]:
# Train it
# .train function requires: input_fn, and steps; returns: a 
# .evaluate function requires: input_fn and steps?; returns a dictionary with metrics
for i in range(10):
    bt_regressor.train(
        input_fn=train_input_fn,
        steps=1000)
    bt_regressor.evaluate(
        input_fn=val_input_fn,
        steps=1000)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data`

INFO:tensorflow:Starting evaluation at 2019-03-11T18:01:09Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./logs/Mon_11_Mar_2019_18:00/model.ckpt-601
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1000]
INFO:tensorflow:Evaluation [400/1000]
INFO:tensorflow:Evaluation [500/1000]
INFO:tensorflow:Evaluation [600/1000]
INFO:tensorflow:Evaluation [700/1000]
INFO:tensorflow:Evaluation [800/1000]
INFO:tensorflow:Evaluation [900/1000]
INFO:tensorflow:Evaluation [1000/1000]
INFO:tensorflow:Finished evaluation at 2019-03-11-18:01:13
INFO:tensorflow:Saving dict for global step 601: average_loss = 0.017430788, global_step = 601, label/mean = 0.47270006, loss = 0.017430788, prediction/mean = 0.4742093
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 601: ./logs/Mon_11_Mar_2019_18:00/model.ckpt-601
INFO:tensor

INFO:tensorflow:Finished evaluation at 2019-03-11-18:01:28
INFO:tensorflow:Saving dict for global step 604: average_loss = 0.017456256, global_step = 604, label/mean = 0.47270006, loss = 0.017456256, prediction/mean = 0.4687566
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 604: ./logs/Mon_11_Mar_2019_18:00/model.ckpt-604
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./logs/Mon_11_Mar_2019_18:00/model.ckpt-604
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Saving checkpoints for 604 into ./logs/Mon_11_Mar_2019_18:00/model.ckpt.
Type is unsupported, or the types of the items don't match fiel

Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:loss = 0.013105624, step = 608
INFO:tensorflow:Saving checkpoints for 608 into ./logs/Mon_11_Mar_2019_18:00/model.ckpt.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Loss for final step: 0.013105624.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-11T18:01:45Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./logs/Mon_11_Mar_2019_18:00/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1

In [17]:
# Make predictions with tf.estimator.Estimator.predict function
# Neccesary argument: input func
# Output A `tf.data.Dataset` object: Outputs of `Dataset` object must have
#            same constraints as below.
#          * features: A `tf.Tensor` or a dictionary of string feature name to
#            `Tensor`. features are consumed by `model_fn`. They should satisfy
#            the expectation of `model_fn` from inputs.
#          * A tuple, in which case the first item is extracted as features.
predictions = bt_regressor.predict(input_fn=test_input_fn)
#predictions1 =  pd.DataFrame([item['predictions'] for item in predictions])
next(predictions)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./logs/Mon_11_Mar_2019_18:00/model.ckpt-609
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'predictions': array([0.2859826], dtype=float32)}

In [18]:
predictions1

NameError: name 'predictions1' is not defined

In [ ]:
preds = predictions
preds[0]

In [ ]:
help(tf.data.Dataset)

In [ ]:
help(tf.estimator.Estimator.predict)